In [1]:
# Change to the repository directory
import os


# Install required packages
!pip install torch torchvision numpy tqdm tensorboard tensorboardX ml-collections medpy SimpleITK scipy h5py opencv-python

# Additional packages that might be needed
!pip install nibabel matplotlib

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 KB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 KB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.7/76.7 KB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.3/156.3 KB 44.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 MB 28.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 55.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 130.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.7/37.7 MB 81.3 MB/s eta 0:00:0000:0100:01
  Created wheel for medpy: filename=MedPy-0.5.2-py3-none-any.whl size=224726 sha256=a269bd30ea46c61d544359de9220213eeb5974232134ca5c2284198c34375821
  Stored in directory: /home/ubuntu/.cache/pi

In [2]:
!pip install scipy tqdm

Defaulting to user installation because normal site-packages is not writeable


In [3]:
!pip install --upgrade matplotlib

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 53.5 MB/s eta 0:00:0000:01:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.0/325.0 KB 77.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.9/113.9 KB 27.4 MB/s eta 0:00:00


In [4]:
# Create necessary directories
import os
from pathlib import Path

# Create model directory for pretrained weights
model_dir = Path('/home/ubuntu/segmentation/TransUNet/model/vit_checkpoint/imagenet21k')
model_dir.mkdir(parents=True, exist_ok=True)

# Create data directory structure
data_dir = Path('/home/ubuntu/segmentation/TransUNet/data/ImageCas')
data_dir.mkdir(parents=True, exist_ok=True)

# Create subdirectories
(data_dir / 'train').mkdir(exist_ok=True)
(data_dir / 'label').mkdir(exist_ok=True)
(data_dir / 'train_processed').mkdir(exist_ok=True)
(data_dir / 'val_processed').mkdir(exist_ok=True)

# Create lists directory
lists_dir = Path('/home/ubuntu/segmentation/TransUNet/lists/lists_ImageCas')
lists_dir.mkdir(parents=True, exist_ok=True)

print("Directory structure created successfully!")

Directory structure created successfully!


In [5]:
import requests
import os

model_path = ('/home/ubuntu/segmentation/TransUNet/model/vit_checkpoint/imagenet21k')
model_url = 'https://storage.googleapis.com/vit_models/imagenet21k/R50+ViT-B_16.npz'
model_file = f'{model_path}/R50+ViT-B_16.npz'

if not os.path.exists(model_file):
    print("Downloading pre-trained model...")
    response = requests.get(model_url, stream=True)
    response.raise_for_status()

    with open(model_file, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)

    print("Model downloaded successfully!")

In [6]:
import os
import glob
import re

dst_label = '/home/ubuntu/imagecas_extracted/labels'
label_files = glob.glob(f'{dst_label}/**/*.label.nii.gz', recursive=True)

print(f"Total found: {len(label_files)}")

# Build expected set of names: "1.label.nii.gz" ... "1000.label.nii.gz"
expected_names = {f"{i}.label.nii.gz" for i in range(1, 1001)}

# Check each file
extra_files = []
for f in label_files:
    fname = os.path.basename(f)
    if fname not in expected_names:
        extra_files.append(f)

print(f"Found {len(extra_files)} unexpected files:")
for f in extra_files:
    print("  ", f)



Total found: 0
Found 0 unexpected files:


In [6]:
import os
import glob
import re

dst_label = '/home/ubuntu/imagecas_extracted/images'
label_files = glob.glob(f'{dst_label}/**/*.img.nii.gz', recursive=True)

print(f"Total found: {len(label_files)}")

# Build expected set of names: "1.label.nii.gz" ... "1000.label.nii.gz"
expected_names = {f"{i}.img.nii.gz" for i in range(1, 1001)}

# Check each file
extra_files = []
for f in label_files:
    fname = os.path.basename(f)
    if fname not in expected_names:
        extra_files.append(f)

print(f"Found {len(extra_files)} unexpected files:")
for f in extra_files:
    print("  ", f)



Total found: 0
Found 0 unexpected files:


In [13]:
#!/usr/bin/env python3
"""Diagnose ImageCAS file structure"""

from pathlib import Path
import SimpleITK as sitk

def diagnose():
    train_dir = Path('/home/ubuntu/imagecas_extracted/images')
    label_dir = Path('/home/ubuntu/imagecas_extracted/labels')
    
    print("="*70)
    print("ImageCAS File Structure Diagnosis")
    print("="*70)
    
    # Check if directories exist
    print(f"\n📁 Directories:")
    print(f"   Train: {train_dir} - {'✓ exists' if train_dir.exists() else '✗ missing'}")
    print(f"   Label: {label_dir} - {'✓ exists' if label_dir.exists() else '✗ missing'}")
    
    if not train_dir.exists() or not label_dir.exists():
        print("\n❌ Directories not found!")
        return
    
    # Get all files
    train_files = sorted(train_dir.glob('*.nii.gz'))
    label_files = sorted(label_dir.glob('*.nii.gz'))
    
    print(f"\n📊 File counts:")
    print(f"   Train files: {len(train_files)}")
    print(f"   Label files: {len(label_files)}")
    
    # Show first 10 files from each
    print(f"\n📄 First 10 train files:")
    for i, f in enumerate(train_files[:10], 1):
        size_mb = f.stat().st_size / (1024 * 1024)
        print(f"   {i}. {f.name:<40} ({size_mb:.1f} MB)")
    
    print(f"\n📄 First 10 label files:")
    for i, f in enumerate(label_files[:10], 1):
        size_mb = f.stat().st_size / (1024 * 1024)
        print(f"   {i}. {f.name:<40} ({size_mb:.1f} MB)")
    
    # Test reading first file
    if train_files:
        print(f"\n🔍 Testing first train file:")
        test_file = train_files[0]
        print(f"   File: {test_file.name}")
        print(f"   Full path: {test_file}")
        
        try:
            img = sitk.ReadImage(str(test_file))
            print(f"   ✓ Successfully read!")
            print(f"   Size: {img.GetSize()}")
            print(f"   Spacing: {img.GetSpacing()}")
            print(f"   Origin: {img.GetOrigin()}")
            print(f"   Pixel type: {img.GetPixelIDTypeAsString()}")
        except Exception as e:
            print(f"   ✗ Failed to read: {e}")
    
    # Check naming patterns
    print(f"\n🔤 Naming patterns:")
    patterns = {
        'has .img.nii.gz': sum(1 for f in train_files if '.img.nii.gz' in f.name),
        'has .label.nii.gz': sum(1 for f in label_files if '.label.nii.gz' in f.name),
        'just .nii.gz': sum(1 for f in train_files if f.name.endswith('.nii.gz') and '.img' not in f.name),
    }
    
    for pattern, count in patterns.items():
        print(f"   {pattern}: {count}")
    
    # Try to match pairs
    print(f"\n🔗 Attempting to match pairs:")
    
    matched = 0
    unmatched_imgs = []
    unmatched_lbls = []
    
    # Get case IDs
    def get_case_id(filename):
        """Extract case ID from filename"""
        name = filename.name
        # Remove all possible suffixes
        for suffix in ['.img.nii.gz', '.label.nii.gz', '.nii.gz']:
            if name.endswith(suffix):
                return name[:-len(suffix)]
        return filename.stem
    
    train_ids = {get_case_id(f): f for f in train_files}
    label_ids = {get_case_id(f): f for f in label_files}
    
    print(f"   Unique train IDs: {len(train_ids)}")
    print(f"   Unique label IDs: {len(label_ids)}")
    
    # Find matches
    common_ids = set(train_ids.keys()) & set(label_ids.keys())
    print(f"   ✓ Matched pairs: {len(common_ids)}")
    
    # Show first 5 matches
    if common_ids:
        print(f"\n   First 5 matched pairs:")
        for i, case_id in enumerate(sorted(common_ids)[:5], 1):
            print(f"      {i}. ID: {case_id}")
            print(f"         Train: {train_ids[case_id].name}")
            print(f"         Label: {label_ids[case_id].name}")
    
    # Show unmatched
    unmatched_train = set(train_ids.keys()) - set(label_ids.keys())
    unmatched_label = set(label_ids.keys()) - set(train_ids.keys())
    
    if unmatched_train:
        print(f"\n   ⚠ Unmatched train files: {len(unmatched_train)}")
        for case_id in sorted(unmatched_train)[:5]:
            print(f"      - {train_ids[case_id].name}")
    
    if unmatched_label:
        print(f"\n   ⚠ Unmatched label files: {len(unmatched_label)}")
        for case_id in sorted(unmatched_label)[:5]:
            print(f"      - {label_ids[case_id].name}")
    
    print(f"\n{'='*70}")

if __name__ == "__main__":
    diagnose()

ImageCAS File Structure Diagnosis

📁 Directories:
   Train: /home/ubuntu/imagecas_extracted/images - ✓ exists
   Label: /home/ubuntu/imagecas_extracted/labels - ✓ exists

📊 File counts:
   Train files: 1000
   Label files: 1000

📄 First 10 train files:
   1. 1.img.nii.gz                             (91.3 MB)
   2. 10.img.nii.gz                            (93.1 MB)
   3. 100.img.nii.gz                           (70.1 MB)
   4. 1000.img.nii.gz                          (76.2 MB)
   5. 101.img.nii.gz                           (67.0 MB)
   6. 102.img.nii.gz                           (85.9 MB)
   7. 103.img.nii.gz                           (63.9 MB)
   8. 104.img.nii.gz                           (90.2 MB)
   9. 105.img.nii.gz                           (91.7 MB)
   10. 106.img.nii.gz                           (69.8 MB)

📄 First 10 label files:
   1. 1.label.nii.gz                           (2.5 MB)
   2. 10.label.nii.gz                          (2.5 MB)
   3. 100.label.nii.gz                 

In [20]:
 !python analyze_hu_range.py \
        --image-dir /home/ubuntu/imagecas_extracted/images \
        --label-dir /home/ubuntu/imagecas_extracted/labels  \
        --num-samples 20

/home/ubuntu/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "
Found 1000 volume pairs
Sampling 20 volumes for analysis

📊 Analyzing volumes...
  [1/20] 568.img.nii.gz... ✅ Vessel HU: 111 ± 158
  [2/20] 762.img.nii.gz... ✅ Vessel HU: 111 ± 122
  [3/20] 765.img.nii.gz... ✅ Vessel HU: 134 ± 167
  [4/20] 693.img.nii.gz... ✅ Vessel HU: 122 ± 157
  [5/20] 469.img.nii.gz... ✅ Vessel HU: 134 ± 140
  [6/20] 709.img.nii.gz... ✅ Vessel HU: 175 ± 185
  [7/20] 662.img.nii.gz... ✅ Vessel HU: 135 ± 148
  [8/20] 560.img.nii.gz... ✅ Vessel HU: 150 ± 191
  [9/20] 872.img.nii.gz... ✅ Vessel HU: 117 ± 143
  [10/20] 220.img.nii.gz... ✅ Vessel HU: 124 ± 124
  [11/20] 829.img.nii.gz... ✅ Vessel HU: 11

In [6]:
# Run preprocessing using local data paths (20 volumes for testing)
os.chdir('/home/ubuntu/segmentation/TransUNet')

# Create output directories in local storage first
os.makedirs('/home/ubuntu/segmentation/TransUNet/data/ImageCas/train_processed_512', exist_ok=True)
os.makedirs('/home/ubuntu/segmentation/TransUNet/data/ImageCas/val_processed_512', exist_ok=True)
os.makedirs('/home/ubuntu/segmentation/TransUNet/data/ImageCas/lists_512', exist_ok=True)

# Run preprocessing with 20 volumes for testing
!python preprocess_imagecas_512.py \
    --image-dir /home/ubuntu/imagecas_extracted/images \
    --label-dir /home/ubuntu/imagecas_extracted/labels \
    --out-train /home/ubuntu/segmentation/TransUNet/data/ImageCas/train_processed_512 \
    --out-test /home/ubuntu/segmentation/TransUNet/data/ImageCas/val_processed_512 \
    --lists-dir /home/ubuntu/segmentation/TransUNet/data/ImageCas/lists_512 \
    --split 0.2 \
    --seed 42 \
   

📦 1000 volumes → train 800, val 200
🔬 Target size: 512×512
📊 HU range: [0, 1000]
⚡ Workers: 8
📁 Compression: ON

⏳ Processing 800 training volumes...
  ✅ 106: 206 slices
  ✅ 101: 206 slices
  ✅ 1000: 233 slices
  ✅ 1: 275 slices
  ✅ 107: 275 slices
  ✅ 102: 257 slices
  ✅ 10: 275 slices
  ✅ 105: 275 slices
  ✅ 109: 275 slices
  ✅ 11: 275 slices
  ✅ 110: 275 slices
  ✅ 111: 275 slices
  ✅ 112: 241 slices
  ✅ 114: 275 slices
  ✅ 113: 275 slices
  ✅ 115: 245 slices
  ✅ 116: 206 slices
  ✅ 118: 275 slices
  ✅ 12: 275 slices
  ✅ 119: 275 slices
  ✅ 121: 275 slices
  ✅ 124: 275 slices
  ✅ 122: 275 slices
  ✅ 123: 275 slices
  ✅ 125: 275 slices
  ✅ 126: 259 slices
  ✅ 127: 261 slices
  ✅ 128: 275 slices
  ✅ 134: 206 slices
  ✅ 131: 247 slices
  ✅ 129: 275 slices
  ✅ 133: 275 slices
  ✅ 135: 275 slices
  ✅ 136: 249 slices
  ✅ 138: 275 slices
  ✅ 137: 275 slices
  ✅ 14: 275 slices
  ✅ 142: 275 slices
  ✅ 140: 275 slices
  ✅ 141: 275 slices
  ✅ 143: 275 slices
  ✅ 145: 223 slices
  ✅ 15: 206 sli

In [5]:
import numpy as np
import SimpleITK as sitk
import matplotlib.pyplot as plt
from pathlib import Path

def visualize_comparison(original_img, original_lbl, processed_dir,
                        slice_indices=None, num_slices=5,
                        hu_min=0, hu_max=4000):
    """Visualize original volumes alongside their preprocessed slices
    
    Updated to match preprocess_imagecas_512_final.py metadata format.

    Args:
        original_img: path to original image volume
        original_lbl: path to original label volume
        processed_dir: directory containing processed slices
        slice_indices: specific slices to visualize (or None for auto)
        num_slices: number of slices if slice_indices is None
        hu_min: HU minimum used in preprocessing (default: 0)
        hu_max: HU maximum used in preprocessing (default: 4000)
    """

    # Load original volumes
    vol = sitk.ReadImage(str(original_img))
    lbl = sitk.ReadImage(str(original_lbl))
    v_arr = sitk.GetArrayFromImage(vol)
    l_arr = sitk.GetArrayFromImage(lbl)

    # Normalize original same way as preprocessing (fixed normalization)
    v_arr_norm = np.clip(v_arr, hu_min, hu_max)
    v_arr_norm = (v_arr_norm - hu_min) / (hu_max - hu_min)

    # Determine which slices to visualize
    total_slices = v_arr.shape[0]
    if slice_indices is None:
        # Auto-select slices with foreground
        fg_slices = np.where(l_arr.sum(axis=(1,2)) > 0)[0]
        if len(fg_slices) >= num_slices:
            slice_indices = fg_slices[np.linspace(0, len(fg_slices)-1, num_slices, dtype=int)]
        else:
            slice_indices = np.linspace(0, total_slices-1, num_slices, dtype=int)

    print(f"Total slices in volume: {total_slices}")
    print(f"Visualizing slices: {slice_indices}")

    # Extract clean case ID (matching final preprocessing)
    def get_case_id(path):
        name = Path(path).name
        if '.img.nii.gz' in name:
            return name.replace('.img.nii.gz', '')
        if '.label.nii.gz' in name:
            return name.replace('.label.nii.gz', '')
        if name.endswith('.nii.gz'):
            return name[:-7]
        return Path(path).stem

    case_id = get_case_id(original_img)
    print(f"Case ID: {case_id}")

    # Create figure
    fig, axes = plt.subplots(len(slice_indices), 6, figsize=(18, 3*len(slice_indices)))
    if len(slice_indices) == 1:
        axes = axes.reshape(1, -1)

    for idx, z in enumerate(slice_indices):
        # Find corresponding processed file (4-digit slice number)
        processed_file = Path(processed_dir) / f"{case_id}_slice{z:04d}.npz"

        if not processed_file.exists():
            # Try 3-digit format as fallback
            processed_file = Path(processed_dir) / f"{case_id}_slice{z:03d}.npz"
            if not processed_file.exists():
                print(f"⚠️  Processed file not found for slice {z}")
                continue

        print(f"✅ Found: {processed_file.name}")

        # Load processed slice
        data = np.load(processed_file)
        img_processed = data['image']
        lbl_processed = data['label']

        # Get processed size for display
        proc_h, proc_w = img_processed.shape

        # Original image
        axes[idx, 0].imshow(v_arr_norm[z], cmap='gray')
        axes[idx, 0].set_title(f'Original Image\nSlice {z} ({v_arr[z].shape[0]}×{v_arr[z].shape[1]})')
        axes[idx, 0].axis('off')

        # Original label
        axes[idx, 1].imshow(l_arr[z], cmap='jet', vmin=0, vmax=1)
        axes[idx, 1].set_title(f'Original Label\nSlice {z}')
        axes[idx, 1].axis('off')

        # Original overlay
        axes[idx, 2].imshow(v_arr_norm[z], cmap='gray')
        axes[idx, 2].imshow(l_arr[z], cmap='jet', alpha=0.5, vmin=0, vmax=1)
        axes[idx, 2].set_title(f'Original Overlay\nSlice {z}')
        axes[idx, 2].axis('off')

        # Processed image
        axes[idx, 3].imshow(img_processed, cmap='gray')
        axes[idx, 3].set_title(f'Processed Image\n({proc_h}×{proc_w})')
        axes[idx, 3].axis('off')

        # Processed label
        axes[idx, 4].imshow(lbl_processed, cmap='jet', vmin=0, vmax=1)
        axes[idx, 4].set_title(f'Processed Label\n({proc_h}×{proc_w})')
        axes[idx, 4].axis('off')

        # Processed overlay
        axes[idx, 5].imshow(img_processed, cmap='gray')
        axes[idx, 5].imshow(lbl_processed, cmap='jet', alpha=0.5, vmin=0, vmax=1)
        axes[idx, 5].set_title(f'Processed Overlay\n({proc_h}×{proc_w})')
        axes[idx, 5].axis('off')

    plt.suptitle(f'Volume: {case_id} | HU range: [{hu_min}, {hu_max}]',
                 fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

    # Show metadata if available (updated keys for final preprocessing)
    meta_file = Path(processed_dir) / f"{case_id}_metadata.npz"
    if meta_file.exists():
        meta = np.load(meta_file, allow_pickle=True)
        print(f"\n📋 Metadata for {case_id}:")
        print(f"   SITK size (x,y,z): {meta['sitk_size']}")
        print(f"   SITK spacing (x,y,z): {meta['sitk_spacing']} mm")
        print(f"   NumPy shape (z,y,x): {meta['numpy_shape']}")
        print(f"   Target size: {meta['target_size']}")
        print(f"   Resize factors (y,x): {meta['resize_factors']}")
        print(f"   Effective spacing (x,y,z): {meta['effective_spacing']} mm")
        print(f"   HU range: [{meta['hu_min']}, {meta['hu_max']}]")
        print(f"   Number of slices: {meta['num_slices']}")
        if 'unique_labels' in meta:
            print(f"   Unique labels: {meta['unique_labels']}")
    else:
        print(f"\n⚠️  Metadata file not found: {meta_file}")


# ============================================================
# RUN VISUALIZATION
# ============================================================
visualize_comparison(
    original_img="/home/ubuntu/imagecas_extracted/images/174.img.nii.gz",
    original_lbl="/home/ubuntu/imagecas_extracted/labels/174.label.nii.gz",
    processed_dir="/home/ubuntu/segmentation/TransUNet/data/ImageCas/train_processed_512",
    slice_indices=None,  # Auto-select slices with foreground
    num_slices=5,
    hu_min=0,      # Match preprocessing settings
    hu_max=1000    # Match preprocessing settings
)

/home/ubuntu/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


RuntimeError: Exception thrown in SimpleITK ImageFileReader_Execute: /work/src/Code/IO/src/sitkImageReaderBase.cxx:91:
sitk::ERROR: The file "/home/ubuntu/imagecas_extracted/images/174.img.nii.gz" does not exist.

In [4]:
!pip install "numpy<2" --break-system-packages


Usage:   
  pip install [options] <requirement specifier> [package-index-options] ...
  pip install [options] -r <requirements file> [package-index-options] ...
  pip install [options] [-e] <vcs project url> ...
  pip install [options] [-e] <local project path> ...
  pip install [options] <archive url/path> ...

no such option: --break-system-packages
